In [3]:
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt



In [4]:
review=pd.read_json('/Users/Paul/Desktop/chinese_restaurants_review/chinese_restaurants_review.json', lines=True)


In [5]:
review[0:3]

,business_id,cool,date,funny,review_id,stars,text,type,useful,user_id
0,-lbN0pSYgK2BRnHNV48PMw,0,2015-05-18,0,PyE0jOhNWfgIBiqMvxnnZA,5,This place is a gem. We had friendly attentive...,review,0,a8S38gZi5r0LAM1UYYP78A
1,-lbN0pSYgK2BRnHNV48PMw,0,2014-11-28,0,751DoGeoBcDMXbFCxQvOoA,5,This is perhaps the closest pho restaurant to ...,review,0,C9z04s9HhssHZ_Rd8QnVsQ
2,-lbN0pSYgK2BRnHNV48PMw,0,2014-06-17,0,FrF2YacZKin85hIN1fz0qQ,4,Happened to stumble upon this little quaint re...,review,1,Xij9XLFMVDczC9THV8luYQ


In [6]:
import nltk
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
ps = PorterStemmer()
def token(x):
    
    x = x.lower()
    x_token = nltk.word_tokenize(x)
    
    # remove stop words
    stopWords = set(stopwords.words('english'))
    x_token = [p for p in x_token if p not in stopWords]
    
     #remove punctuations
    table=str.maketrans(dict.fromkeys(string.punctuation))
    x_token = [p.translate(table) for p in x_token ]
      
        
    # after removing punctuation, some token may become ' '
    return [p for p in x_token if p != '']


In [8]:
# a test for the funciton token()

x='Hello! I like to play^^ Basketball:)'
x = x.lower()
x = nltk.word_tokenize(x)
stopWords = set(stopwords.words('english'))
y = [p for p in x if p not in stopWords]
table=str.maketrans(dict.fromkeys(string.punctuation))
z = [q.translate(table) for q in y ]

print(x)
print(y)
print(z)

['hello', '!', 'i', 'like', 'to', 'play^^', 'basketball', ':', ')']
['hello', '!', 'like', 'play^^', 'basketball', ':', ')']
['hello', '', 'like', 'play', 'basketball', '', '']


In [9]:
# drop irrelavant information for our purpose

review=review.drop(['business_id', 'cool', 'date', 'funny', 'review_id', 'type', 'user_id'], axis=1)

In [10]:
review[0:3]

,stars,text,useful
0,5,This place is a gem. We had friendly attentive...,0
1,5,This is perhaps the closest pho restaurant to ...,0
2,4,Happened to stumble upon this little quaint re...,1


In [11]:
# get token for each review item

begin = datetime.datetime.now()
review['token'] = review['text'].apply(lambda x: token(x))
print ('Total time spent:', datetime.datetime.now() - begin)
review.head()



Total time spent: 0:18:43.902904


,stars,text,useful,token
0,5,This place is a gem. We had friendly attentive...,0,"[place, gem, friendly, attentive, service, foo..."
1,5,This is perhaps the closest pho restaurant to ...,0,"[perhaps, closest, pho, restaurant, port, cred..."
2,4,Happened to stumble upon this little quaint re...,1,"[happened, stumble, upon, little, quaint, rest..."
3,4,Very good food for decent prices. The atmosphe...,0,"[good, food, decent, prices, atmosphere, nice,..."
4,3,The food was very fresh and quite tasty. The m...,1,"[food, fresh, quite, tasty, mango, salad, nice..."


In [12]:
# drop the 'text' column
review=review.drop('text', axis=1)
review.head()

,stars,useful,token
0,5,0,"[place, gem, friendly, attentive, service, foo..."
1,5,0,"[perhaps, closest, pho, restaurant, port, cred..."
2,4,1,"[happened, stumble, upon, little, quaint, rest..."
3,4,0,"[good, food, decent, prices, atmosphere, nice,..."
4,3,1,"[food, fresh, quite, tasty, mango, salad, nice..."


In [10]:
review.to_json('/Users/Paul/Desktop/chinese_restaurants_review/review_processed.json', orient='records', lines=True)

In [13]:
from gensim.models import word2vec

C:\Users\Paul\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [14]:
num_features = 300   # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10         # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [16]:
import time
begin = time.time()
w2v_model = word2vec.Word2Vec(review.token, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)
print ('Total time spent:', time.time() - begin)

Total time spent: 218.11289191246033


In [9]:
w2v_model.similarity('tasty', 'good')

NameError: name 'w2v_model' is not defined

In [106]:
w2v_model.save('/Users/Paul/Desktop/chinese_restaurants_review/w2v_model')

In [13]:
w2v_model=word2vec.Word2Vec.load('/Users/Paul/Desktop/chinese_restaurants_review/w2v_model')

In [2]:
def Average(token):
    
    # Here 300 is the number of neurals in the hidden layer
    feature_vec = np.zeros((num_features,), dtype="float32")
    
    n_words = 0
    # we have trained a word2vec model named 'w2v_model'
    
    # make a set of words we have learned in model
    word_set = set(w2v_model.wv.index2word)
    
    # only arverage the words that has certain correlation/similarity with the judgement words "good" or "bad". The others are ignored.
    for x in token:
        if x in word_set and (w2v_model.similarity('good', x)>=0 or w2v_model.similarity('bad', x)>=0):
            n_words+= 1
            feature_vec += w2v_model[x]
    
    return np.divide(feature_vec,float(n_words))

In [24]:
import time

In [27]:
begin = time.time()
review['w2v_feature'] =review['token'].apply(lambda x: Average(x))
print ('Total time spent:', time.time() - begin)

Total time spent: 1567.4630041122437


In [28]:
# removing rows that have Nan value for 'w2v_feature'
ix = []
for i in range(review.shape[0]):
    if np.any(np.isnan(review.w2v_feature.iloc[i])): ix.append(i)
review = review.drop(ix, axis= 0)

In [ ]:
review.to_json('/Users/Paul/Desktop/chinese_restaurants_review/review_w2v.json', orient='records', lines=True)